In [13]:
import pandas as pd
from data_pipeline import data_pipeline, restore_authors, dot_problem_solver
from math import ceil

In [14]:
file_rzepa = 'Rzeczpospolita 2015-2022.xlsx'
file_gpc = 'excels/Gazeta Polska Codziennie 2015-2022.xlsx'
file_newsweek = 'excels/Newsweek.xlsx'
file_wprost = "excels/wprost-2015.xlsx"
cols_rzepa = ['title', 'text', 'source', 'author','link','department','imgage','image_text']
cols_gpc = ['title', 'text', 'source', 'author','link','department','imgage']
cols_newsweek = ['title', 'text']

In [15]:
data = pd.read_excel(file_rzepa)

In [16]:
data['source'].value_counts()

Rzeczpospolita    216163
Name: source, dtype: int64

In [17]:
# Newsweek
#data.rename(columns={'title_main': 'title', 'author 1': 'author', "Wydanie": "department"}, inplace=True)

In [18]:
data = data_pipeline(data, cols_rzepa)
data = restore_authors(data) # Rzepa
data['text'] = data['text'].apply(dot_problem_solver)

/home/hombre/Code/Climate Project/DataPrep/Rzepa/data_pipeline.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].fillna('None')
/home/hombre/Code/Climate Project/DataPrep/Rzepa/data_pipeline.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: re.split('(?i)©℗\s*Więcej\s*na', x)[0])
/home/hombre/Code/Climate Project/DataPrep/Rzepa/data_pipeline.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [19]:
data.to_csv("rzepa_all.csv", index=False)

In [ ]:
!mkdir chunks

In [ ]:
def chunkit(df, chunk_size= 10000):
    num_chunks = ceil(len(df) / chunk_size)
    for i in range(num_chunks):
        yield df[i*chunk_size:(i + 1) * chunk_size]

for i, chunk in enumerate(chunkit(data)): chunk.to_csv(f"chunks/rzepa_{i}.csv", index=False)

In [ ]:
!rm -r chunks